In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

In [5]:
df=pd.read_csv(r'E:\user\hyper star qualifying task\source\data\edited\df1.csv')

In [6]:
df.head()

,StoreID,Date,Sales,year,month,day,day_of_week,week
0,1,2020-07-01,29559.916214,2020,7,1,2,27
1,1,2020-07-02,36765.847944,2020,7,2,3,27
2,1,2020-07-03,31261.577192,2020,7,3,4,27
3,1,2020-07-04,26516.164081,2020,7,4,5,27
4,1,2020-07-05,31614.797517,2020,7,5,6,27


# pipeline processing

#### 1 getting the desired data

1. determine the store id

2. get the store id dataset

3. group it by months' sales and prepare it


#### 2 prepare it for forecasting

1. shifting the values by the desired

2. splitting the dataset into train and test

.


`1 getting the desired data`

In [33]:
store_id=1
class store_id(BaseEstimator, TransformerMixin):
    def __init__(self,ID=store_id):
         
        self.ID = ID

    def fit(self,X,y=None):
        
        return self
        
    
    def transform(self,x):
    
        x=x[x['StoreID']==self.ID][:-1] 
        
        return x

In [25]:
class prepare(BaseEstimator, TransformerMixin):
    
    def fit(self,X,y=None):
        
        return self
    
    def transform(self,x):
        from sklearn.preprocessing import MinMaxScaler
        sc_out = MinMaxScaler(feature_range=(0, 1))
        month_sale=x.groupby(['year','month'])['Sales'].sum().values
        length=len(x.groupby(['year','month'])['Sales'].sum().index)
        index=[]
    
        for i in range(length):
            year=x.groupby(['year','month'])['Sales'].sum().index[i][0]
            month=x.groupby(['year','month'])['Sales'].sum().index[i][1]
            val=str(year)+'-'+str(month)
            index.append(val)
    
        new_df=x.groupby(['year','month'])['Sales'].sum()
        new_df.index=index
        values=new_df.values
        new_df.index=pd.to_datetime(new_df.index)
        df=pd.DataFrame(index=new_df.index,columns=['sales'])
        df['sales']=values
        df['sales']=sc_out.fit_transform(df[['sales']])
    
        return df

`2 prepare it for forecasting` 

In [28]:
step=-1

class shift(BaseEstimator, TransformerMixin):
    
    def __init__(self,steps=step):
        
        self.steps=steps

    def fit (self,X,y=None):
        
        return self
    
    
    def transform(self,x):
  
        df_pre=x.copy()
        df_pre['actual']=x['sales'].shift(self.steps)
    
        return df_pre

In [24]:
fra=0.8

class train_test_split(BaseEstimator, TransformerMixin):
    
    def __init__(self,fraction=fra):
        
        self.fraction=fraction

    def fit(self,X,y=None):
        
        return self
    
    def transform(self,x):

        train_size=int(len(x)*self.fraction)
        test_size = int(len(x)) - train_size
    
        y=x['actual']
        y.index=x.index
        y.rename({0:'sales_for_next_day'},inplace= True)
        train_y = y[:train_size]
        test_y = y[train_size:]
    
        return train_y, test_y

In [11]:
from sklearn.pipeline import Pipeline

In [34]:
pipe=Pipeline([
    ('ID',store_id()),
    ('prepare',prepare()),
    ('shift',shift()),
    ('train test split',train_test_split())
])

In [35]:
train_y,test_y=pipe.fit_transform(df)

In [36]:
train_y

2020-07-01    0.000000
2020-08-01    0.103671
2020-09-01    0.210662
2020-10-01    0.031075
2020-11-01    0.229593
2020-12-01    0.244542
2021-01-01    0.304574
2021-02-01    0.392912
2021-03-01    0.294765
2021-04-01    0.293776
2021-05-01    0.253601
2021-06-01    0.508462
2021-07-01    0.547457
2021-08-01    0.586830
2021-09-01    0.589019
2021-10-01    0.668950
2021-11-01    0.749474
2021-12-01    0.424679
2022-01-01    0.408926
Name: actual, dtype: float64

with this prepared dataset, you can go on modeling (modeling section)